# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **K means model** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Integrantes**:
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto
- Ximena Isaac Horta
- Alberto Renteria Camacho

In [1]:
import findspark
findspark.init()

In [2]:
SPARK_ID = "91c5715ce4dd"

#### Spark Conexion

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-K-means") \
    .master(f"spark://{SPARK_ID}:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 01:37:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Extraction of data

In [4]:
data_path = "/home/jovyan/notebooks/data/parquet/"

raw_df = spark.read \
    .parquet(data_path)

print("Rows:", raw_df.count(), "Cols:", len(raw_df.columns))
raw_df.show(10)

Rows: 1580 Cols: 8


+----+--------------------+-------+---------+------+--------------------+-------------+--------------------+
| key|               value|  topic|partition|offset|           timestamp|timestampType|           value_str|
+----+--------------------+-------+---------+------+--------------------+-------------+--------------------+
|NULL|[7B 22 74 77 65 6...|tweet-2|        0|     4|2025-05-07 18:35:...|            0|{"tweet_id": "5db...|
|NULL|[7B 22 74 77 65 6...|tweet-2|        0|     5|2025-05-07 18:35:...|            0|{"tweet_id": "d7b...|
|NULL|[7B 22 74 77 65 6...|tweet-2|        0|     6|2025-05-07 18:35:...|            0|{"tweet_id": "621...|
|NULL|[7B 22 74 77 65 6...|tweet-2|        0|     7|2025-05-07 18:35:...|            0|{"tweet_id": "707...|
|NULL|[7B 22 74 77 65 6...|tweet-2|        0|     8|2025-05-07 18:35:...|            0|{"tweet_id": "bc8...|
|NULL|[7B 22 74 77 65 6...|tweet-2|        0|     9|2025-05-07 18:35:...|            0|{"tweet_id": "738...|
|NULL|[7B 22 74 77 

#### Data preparation

In [5]:
from equipo.spark_utils import SparkUtils
from pyspark.sql.functions import from_json

headers = [
        ("tweet_id", "string"),
        ("user_id", "integer"),
        ("timestamp", "string"),
        ("text", "string"),
        ("hashtags", "string"),
        ("mentions", "string"),
        ("retweet_count", "integer"),
        ("favorite_count", "integer"),
        ("reply_count", "integer"),
        ("quote_count", "integer"),
        ("views", "integer")
]

schema = SparkUtils.generate_schema([(head[0], head[1]) for head in headers])

tweets_df = raw_df.select(from_json(raw_df.value_str, schema).alias("data")).select("data.*")

print("Rows:", tweets_df.count(), "Cols:", len(tweets_df.columns))
tweets_df.printSchema()
tweets_df.show(10)

Rows: 1580 Cols: 11
root
 |-- tweet_id: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- text: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- mentions: string (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- reply_count: integer (nullable = true)
 |-- quote_count: integer (nullable = true)
 |-- views: integer (nullable = true)



+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+-----+
|            tweet_id|user_id|           timestamp|                text|            hashtags|            mentions|retweet_count|favorite_count|reply_count|quote_count|views|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+-----+
|5dbb99f0-e6c9-4f1...|      2|2024-11-04T17:08:...|Distinctio ration...|                  []|["camila48","sote...|          412|           259|          1|         25|10413|
|d7b832b2-88a8-423...|      2|2025-02-09T22:14:...|Minima labore rep...|["magnam","vel","...|        ["kmaestas"]|          438|           165|         46|         36|15013|
|6219028b-becf-4eb...|      2|2025-01-04T16:59:...|Est asperiores qu...|["consequatur","s...| ["esquiveljoaquin"]|           53|  

### Assemble the features into a single vector column

In [6]:
from pyspark.ml.feature import VectorAssembler

numeric_cols = [
    "retweet_count", "favorite_count", "reply_count", "quote_count", "views"
]

assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
assembled_df = assembler.transform(tweets_df)

In [ ]:
print("Rows:", assembled_df.count(), "Cols:", len(assembled_df.columns))

---
# KMeans at the same time

#### Initialize KMeans

In [7]:
from pyspark.ml.clustering import KMeans

k_values = [2, 5, 10, 15, 20]
kmeans = [KMeans().setK(k).setSeed(19) for k in k_values]

#### Training

In [ ]:
models = [k.fit(assembled_df) for k in kmeans]

25/05/08 01:45:04 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/08 01:45:04 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


#### Predictions

In [ ]:
predictions = [model.transform(assembled_df) for model in models]

#### Evaluate model

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

for i in range(len(k_values)):
    print(f"\n K values = {k_values[i]}")
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions[i])
    print(f"Silhouette score: {silhouette}")

In [ ]:
for i in range(len(k_values)):
    print(f"\n K values = {k_values[i]}")
    print("Cluster Centers: ")
    for center in models[i].clusterCenters():
        print(center)

In [11]:
sc.stop()